In [1]:
from tensorflow import keras
keras.__version__

'2.4.0'

To start using zoo.orca, we need to first initialize orca context. Here we specify local or distributed mode. In this example, we choose the local mode.

In [2]:
from zoo.orca import init_orca_context, stop_orca_context
from zoo.orca import OrcaContext

OrcaContext.log_output = True # recommended to set it to True when running Analytics Zoo in Jupyter notebook (this will display terminal's  stdout and stderr in the Jupyter notebook).

cluster_mode = "local"

if cluster_mode == "local":  
    init_orca_context(cluster_mode="local", cores=4) # run in local mode
elif cluster_mode == "k8s":  
    init_orca_context(cluster_mode="k8s", num_nodes=2, cores=2) # run on K8s cluster
elif cluster_mode == "yarn":  
    init_orca_context(cluster_mode="yarn-client", num_nodes=2, cores=2) # run on Hadoop YARN cluster

Initializing orca context
Current pyspark location is : /intern/spark/spark-2.4.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /home/zhenhao/analytics-zoo/zoo/target/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-dist-all/lib/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-jar-with-dependencies.jar pyspark-shell 


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/zhenhao/analytics-zoo/zoo/target/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-dist-all/lib/analytics-zoo-bigdl_0.12.1-spark_2.4.3-0.10.0-SNAPSHOT-jar-with-dependencies.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/intern/spark/spark-2.4.3-bin-hadoop2.7/jars/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-02-04 17:02:10 WARN  Utils:66 - Your hostname, intern01 resolves to a loopback address: 127.0.1.1; using 10.239.44.107 instead (on interface eno1)
2021-02-04 17:02:10 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2021-02-04 17:02:10 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2021-02-04 17:02:11 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
cls.getname: com.intel.analytics.bigdl.python.api.Sample
BigDLBasePickler registering: bigdl.util.common  Sample
cls.getname: com.intel.analytics.bigdl.python.api.EvaluatedResult
BigDLBasePickler registering: bigdl.util.common  EvaluatedResult
cls.getname: com.intel.analytics.bigdl.python.api.JTensor
BigDLBasePickler registering: bigdl.util.common  JTensor
cls.getname: com.intel.analytics.bigdl.python.api.JActivity
BigDLBasePickler registering: bigdl.util.common  JActivity
Successfully got a 


User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_INIT_WAIT=2048
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NEXT_WAIT=1024
   KMP_NUM_L

# Text generation with LSTM
This notebook contains the code samples found in Chapter 8, Section 1 of Deep Learning with Python. Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

-----------------------------------------

[...]

## Implementing character-level LSTM text generation
Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the English language.

## Preparing the data
Let's start by downloading the corpus and converting it to lowercase:

In [3]:
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


Next, we will extract partially-overlapping sequences of length maxlen, one-hot encode them and pack them in a 3D Numpy array `x` of shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot encoded characters that come right after each extracted sequence.

In [4]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [5]:
x.shape

(200278, 60, 57)

In [6]:
y.shape

(200278, 57)


## Building the network
Our network is a single LSTM layer followed by a Dense classifier and softmax over all possible characters. But let us note that recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in recent times. Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [5]:
from tensorflow.keras import layers

def build_model(config):
    model = keras.Sequential()
    model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
    model.add(layers.Dense(len(chars), activation='softmax'))
    optimizer = keras.optimizers.RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

We also create a `data_creator` function that help us import the data into the orca estimator.

In [6]:
import tensorflow as tf

def train_data_creator(config):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.repeat()
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(config["batch_size"])
    return dataset

Training the language model and sampling from it
Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, and draw a character index from it (the "sampling function"):

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of temperature in the sampling strategy.

In [8]:
from zoo.orca.learn.tf2 import Estimator
import random
import sys

batch_size=128
est = Estimator.from_keras(model_creator=build_model,
                           config={},
                           workers_per_node=1,
                           verbose=0)

2021-02-04 17:02:48,985	INFO resource_spec.py:212 -- Starting Ray with 17.29 GiB memory available for workers and up to 8.66 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '10.239.44.107', 'redis_address': '10.239.44.107:35472', 'object_store_address': '/tmp/ray/session_2021-02-04_17-02-48_984157_17398/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2021-02-04_17-02-48_984157_17398/sockets/raylet', 'webui_url': None, 'session_dir': '/tmp/ray/session_2021-02-04_17-02-48_984157_17398'}
(pid=17535) 2021-02-04 17:02:50.042386: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
(pid=17535) 2021-02-04 17:02:50.042423: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(pid=17535) 2021-02-04 17:02:50.917144: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
(pid

In [9]:
from zoo.orca.data import XShards

def transform_to_dict(data):
    return {"x": data[0]}


for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    stats = est.fit(train_data_creator, 
                    epochs=1,
                    batch_size=batch_size,
                    steps_per_epoch=200278 // batch_size,
                    verbose=0)
    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            sample_shards = XShards.partition({"x": sampled}, num_shards=1)
            #sample_shards = sample_shards.transform_shard(transform_to_dict)

            preds = est.predict(sample_shards).collect()
            
            next_index = sample(preds[0]['prediction'][0], temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
    

epoch 1
(pid=17535) WARNING:tensorflow:From /home/zhenhao/.local/lib/python3.7/site-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
(pid=17535) Instructions for updating:
(pid=17535) Use `tf.data.Iterator.get_next_as_optional()` instead.
--- Generating with seed: "heavy, difficult, dangerous thoughts, and
a tempo of the gal"
------ temperature: 0.2
heavy, difficult, dangerous thoughts, and
a tempo of the gal

2021-02-04 17:05:14,653	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:14,666	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:14,811	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:14,813	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:14,905	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:14,905	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:14,906	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config 

1/1 [==============================] - 0s 185us/step
l

2021-02-04 17:05:16,628	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:16,634	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:16,656	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:16,700	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:17,444	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:17,510	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:05:17,568	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config 

1/1 [==============================] - 0s 181us/step
 

...

1/1 [==============================] - 0s 151us/step
(pid=17535) WARNING:tensorflow:11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f784a73d320> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
n

2021-02-04 17:06:56,948	WARNING worker.py:1072 -- WARNING: 149 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2021-02-04 17:06:56,961	WARNING worker.py:1072 -- WARNING: 150 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2021-02-04 17:06:56,977	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:06:57,259	WARNING worker.py:1072 -- WARNING: 151 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644

1/1 [==============================] - 0s 1ms/step
(pid=17535) WARNING:tensorflow:11 out of the last 11 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7f7849b86f80> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
d

2021-02-04 17:06:59,856	WARNING worker.py:1072 -- WARNING: 152 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2021-02-04 17:06:59,864	WARNING worker.py:1072 -- WARNING: 153 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2021-02-04 17:06:59,936	WARNING worker.py:792 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2021-02-04 17:07:00,150	WARNING worker.py:1072 -- WARNING: 154 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644

KeyboardInterrupt: 

In [ ]:
est.shutdown()
#stop_orca_context()